In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm
import json

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

def file_to_string(fileName,start,end):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string.replace('$startDate$',start).replace('$endDate$',end)

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def add_conditionals(df):
    df['Date']=pd.to_datetime(df['Date'])
    test_group = np.array(['364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275']).astype(int)
    df['TestGroup'] = (df['Employee'].isin((test_group))).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Constant'] =1
    return df
    
def overlaid_histogram(df, seriesNameList, title, x_axisTitle, histnorm='',bins=100):
    import plotly.graph_objects as go
    # Build histograms
    df = df.copy()
    fig = go.Figure()
    for series in seriesNameList:
        fig.add_trace(go.Histogram(x=df[series],histnorm=histnorm,name=series, nbinsx=bins ))

    # Overlay histograms
    fig.update_layout(
        barmode='overlay',
        xaxis_title_text=x_axisTitle,
        yaxis_title_text=histnorm,
        title = title)
    # Reduce opacity to see both histograms
    fig.update_traces(opacity=0.6)
    fig.show()

# Run query
Uses a fuzzy join to see how many helix searchs/expert assist messages an expert had durring the time where they were taking a call.

In [ ]:
dfp = run_query(
    file_to_string(
        fileName='AHT And Sales - With Helix and Expert Assist Fuzzy Join.SQL',
        start='2023-08-18',
        end = '2023-11-02'
    ))
display(dfp.describe(),dfp.head())

In [ ]:
df = dfp.copy()
conds = [np.logical_and(df['HelixSearches']>=1,df['ExpertAssistSearches']>=1),df['ExpertAssistSearches']>=1,df['HelixSearches']>=1]
choices=['Both','Expert Assist','Helix']
df['Tool Used'] = np.select(condlist=conds,choicelist=choices,default='None')
# display(df.head())
percentile_List = [.1,.2,.3,.4,.5,.6,.7,.8,.9] #[.5,.75,.9,.95]
describe_df = pd.DataFrame(df['HandleTime'].describe(percentiles=percentile_List)).rename({'HandleTime':'Total With or Without Tools'},axis=1)
for tool in ['None','Expert Assist','Helix','Both']:
        describe_df=describe_df.join(pd.DataFrame(df[df['Tool Used']==tool]['HandleTime'].describe(percentiles=percentile_List)).rename({'HandleTime':tool},axis=1))
        # px.histogram(df[df['Tool Used']==tool],color='Tool Used',x='HandleTime',range_x=[0,10000],nbins=10,histnorm='percent').show()
del tool
        
        
display(describe_df)
del df, conds, choices, percentile_List, describe_df

In [ ]:
df =dfp.copy()
df = add_conditionals(df)
print(df.columns)
df[[
    'Date', 'Employee', 'HandleTime', 'Offered',
    'Accepted','HelixSearches', 'ExpertAssistSearches',
    'TestGroup','AfterExpertAssistLaunch'
    ]].to_excel('AHT And Sales - With Helix and Expert Assist Fuzzy Join.xlsx')
del df

In [ ]:
df = dfp.copy()
conds = [np.logical_and(df['HelixSearches']>=1,df['ExpertAssistSearches']>=1),df['ExpertAssistSearches']>=1,df['HelixSearches']>=1]
choices=['Both','Expert Assist','Helix']
df['Tool Used'] = np.select(condlist=conds,choicelist=choices,default='None')
px.strip(df[df['Tool Used'].isin(['Expert Assist','Helix'])].copy(),y='HandleTime',hover_data=['Date'],color='Tool Used').show()
px.scatter(df[df['Tool Used'].isin(['Expert Assist','Helix'])].copy(),y='HandleTime',x='Date',color='Tool Used').show()
df = df.pivot(columns='Tool Used',values='HandleTime')

overlaid_histogram(df,
    seriesNameList=['Helix','Expert Assist'],
    x_axisTitle='Handle Time',title='AHT by Tool Use',
    bins=30,
    histnorm='percent')

del df, conds, choices